In [10]:
#Imports the Spotipy library, which is a Python client for the Spotify Web API.
import spotipy
#import the class from Spotipy for authentication.
from spotipy.oauth2 import SpotifyClientCredentials
#Imports the ReadTimeout exception from the requests library to handle timeout errors.
from requests.exceptions import ReadTimeout
#Imports the SpotifyException from Spotipy to handle Spotify-specific errors.
from spotipy.exceptions import SpotifyException
#Imports the time module to use functions like sleep for pausing execution.
import time

In [11]:
client_id = '3550a08d516640fbba2f0b46300f5221'
client_secret = '8c43651820ac426a91f4a64b44b023c9'

In [12]:
# Initialize Spotify client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Retrieve available genres for recommendations
genres = (sp.recommendation_genre_seeds())['genres']


In [13]:
markets = ['US', 'GB', 'AU', 'CA', 'NZ']#marakets of the songs 
#Using the function genre,market all tracks into the list
def get_tracks_for_genre_and_market(genre, market):
    track_data = []
    results = sp.search(q=f'genre:{genre}', limit=50, type='track', market=market)#search is the parametre for the sp with search to the genres with diffrent tracks with limit of 50
    
    while results and len(track_data) < 50000:
        track_ids = [item['id'] for item in results['tracks']['items']]#Track Id's iteam  name 
        if not track_ids:
            break
        
        # Fetch audio features for the tracks
        audio_features = sp.audio_features(track_ids) #is autio_features it was support the diffrenent features by the ferching the parametres
        
        for i, item in enumerate(results['tracks']['items']):#returns an iterator that produces pairs of an index and the corresponding item from the iterable.
            track = item
            features = audio_features[i] if audio_features[i] else {}
            track_info = {
                'Name': track['name'],
                'Artist': track['artists'][0]['name'],
                'genres': genre,#The genre of the track.
                'acousticness': features.get('acousticness', None),#Likelihood of the track being acoustic.
                'danceability': features.get('danceability', None),# Suitability for dancing.
                'duration_ms': track['duration_ms'],#Duration of the track in milliseconds.
                'energy': features.get('energy', None),#Intensity and activity of the track.
                'instrumentalness': features.get('instrumentalness', None),#Likelihood of the track being instrumental.
                'liveness': features.get('liveness', None),# Presence of an audience in the recording.
                'loudness': features.get('loudness', None),#Overall loudness of the track.
                'speechiness': features.get('speechiness', None),#Presence of spoken words.
                'tempo': features.get('tempo', None),#Tempo of the track in beats per minute (BPM).
                'valence': features.get('valence', None),#Musical positiveness conveyed.
                'popularity': track['popularity'],#Popularity of the track.
                'mode':features.get('mode', None), # Represents the musical mode (Major or Minor). It will be 1 for Major and 0 for Minor.
                'key':features.get('key', None),  #  Represents the musical key,It is an integer from 0 to 11 corresponding to musical notes (C = 0, C♯/D♭ = 1, etc)
                'album_image_url': track['album']['images'][0]['url'],  # Album image URL
                'track_url': track['external_urls']  # Track URL
            }
            track_data.append(track_info)
        
            if len(track_data) >= 50000:
                break
        
        results = sp.next(results) if results.get('next') else None # checking the all geners with next 
    
    return track_data
#Empty list to stroe the data
all_tracks = []
try:
    for genre in genres:
        for market in markets:
            track_data = get_tracks_for_genre_and_market(genre, market)
            all_tracks.extend(track_data)
            print(f"Tracks collected: {len(all_tracks)} from market {market}")#Track collection how many tracks all avaliable 
            
            if len(all_tracks) >= 50000:#Data collecting in 10000 rows
                all_tracks = all_tracks[:50000]#After 10000 break it
                break
except (ReadTimeout, SpotifyException) as e:
    print(f"Error occurred: {e}")
    time.sleep(5)  # Pause before retrying to avoid rate limit issues

print(f"Total tracks collected: {len(all_tracks)}")

Tracks collected: 50 from market US
Tracks collected: 100 from market GB
Tracks collected: 150 from market AU
Tracks collected: 200 from market CA
Tracks collected: 250 from market NZ
Tracks collected: 300 from market US
Tracks collected: 350 from market GB
Tracks collected: 400 from market AU
Tracks collected: 450 from market CA
Tracks collected: 500 from market NZ
Tracks collected: 550 from market US
Tracks collected: 600 from market GB
Tracks collected: 650 from market AU
Tracks collected: 700 from market CA
Tracks collected: 750 from market NZ
Tracks collected: 800 from market US
Tracks collected: 850 from market GB
Tracks collected: 900 from market AU
Tracks collected: 950 from market CA
Tracks collected: 1000 from market NZ
Tracks collected: 1050 from market US
Tracks collected: 1100 from market GB
Tracks collected: 1150 from market AU
Tracks collected: 1200 from market CA
Tracks collected: 1250 from market NZ
Tracks collected: 1300 from market US
Tracks collected: 1350 from mark

In [14]:
#import the pandas for data conversion
import pandas as pd
#converted the data in Dataframe
df=pd.DataFrame(all_tracks)
#saved the data in the csv file
df.to_csv('Genres_url.csv',index=False)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27950 entries, 0 to 27949
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              27950 non-null  object 
 1   Artist            27950 non-null  object 
 2   genres            27950 non-null  object 
 3   acousticness      27950 non-null  float64
 4   danceability      27950 non-null  float64
 5   duration_ms       27950 non-null  int64  
 6   energy            27950 non-null  float64
 7   instrumentalness  27950 non-null  float64
 8   liveness          27950 non-null  float64
 9   loudness          27950 non-null  float64
 10  speechiness       27950 non-null  float64
 11  tempo             27950 non-null  float64
 12  valence           27950 non-null  float64
 13  popularity        27950 non-null  int64  
 14  mode              27950 non-null  int64  
 15  key               27950 non-null  int64  
 16  album_image_url   27950 non-null  object